In [1]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')
import os
import pandas as pd
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import shapefile
from shapely.geometry  import Point, LineString, Polygon, MultiPolygon

from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm
import glob

from django.db.models import Q, Sum, Avg

/home/tserekh/jupnb/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import json

In [3]:
from main_map.models import Organization,OrganizationNatClass, House, PredictedSquare

NameError: name 'Proj' is not defined

In [2]:
from pyproj import Proj, transform

PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

ModuleNotFoundError: No module named 'pyproj'

In [5]:
half_side = 5000

In [6]:
lat, lon = 55.684067, 37.571067

In [7]:
# x, y = transform(PROJ4326, PROJ3857, lon, lat)

In [8]:
def calc_place(row,x,y,nat_class):

    nearest_nat_classes_euclid = row['nearest_nat_classes_euclid']

    dic = json.loads(nearest_nat_classes_euclid)
    distances = np.array(dic[nat_class])
    R = ((x - row['x'])**2 + (y - row['y'])**2)**0.5
    place = sum(distances<R) + 1
    return place

def get_place_features(x,y):
    nat_class = 'Магазин бытовой техники и электроники'
    filter_dic = {
                'x__gt' : x - half_side,
                'y__gt' : y - half_side,
                'x__lt' : x + half_side,
                'y__lt' : y + half_side,
    }
    
    features = ['flat_num', 'x', 'y', 'nearest_nat_classes_euclid']
    objects = House.objects.filter(**filter_dic).values().values_list(*features)
    df = pd.DataFrame(list(objects.values()), columns=features)
    if len(df)==0:
        return {}

    df['place'] = df.apply(lambda row: calc_place(row, x, y, nat_class), axis=1)
    
    gb = df.groupby('place')['flat_num'].sum()
    features_dic = {}
    places = range(1,12)
    for place in places:
        
        values = gb[gb.index==place]
        if len(values)==0:
            value = 0
        else:    
            value = values.iloc[0]
        features_dic[nat_class +'_'+ str(place)] = value

    return features_dic

In [ ]:
el.x, el.y = transform(PROJ4326, PROJ3857, el.lon, el.lat)

In [ ]:
objects = PredictedSquare.objects.filter(machine_features__isnull=True).order_by('x')
print(objects.count())
for object_ in tqdm(objects):
    features_dic = get_place_features(object_.x,object_.y)
    if len(features_dic)==0:
        object_.delete()
    else:
        object_.machine_features = json.dumps(features_dic)
        object_.save()

371497


 49%|████▉     | 182414/371573 [12:31:50<12:59:39,  4.04it/s]

In [ ]:
# objects = OrganizationNatClass.objects.filter(machine_features__isnull=True,nat_class='Магазин бытовой техники и электроники')
# print(objects.count())
# for object_ in tqdm(objects):
#     features_dic = get_place_features(object_.x,object_.y)
#     if len(features_dic)==0:
#         object_.delete()
#     else:
#         object_.machine_features = json.dumps(features_dic)
#         object_.save()